In [ ]:
import pandas as pd
import numpy as np

##  Sales volume data for the estimation of the power consumption by shs usage


In [ ]:

shs_sales_volumes = pd.read_csv('daten/shs_sales_volumes.csv', comment='#')
# compute the average of the product categories 5 to 7
shs_sales_volumes['tot_5-7'] = shs_sales_volumes[['5', '6', '7']].sum(axis=1)

shs_power_categories = pd.read_csv('daten/shs_power_per_product_categories.csv', comment='#')

# compute the average power for each category
shs_power_categories['power_av'] = shs_power_categories[['power_low', 'power_high']].mean(axis=1)

# select only the average power for the product categories 5 to 7
cat_power_av_5_7 = shs_power_categories['power_av'].loc[shs_power_categories.category >= 5].values
# compute the average power per region (multiply the number units sold per product categories by the average power of the respective category)
power_av_5_7 = shs_sales_volumes[['5', '6', '7']].values * cat_power_av_5_7
# compute the average unit power per region (divide the sum over the categories by the total unit sold)
shs_sales_volumes['weighted_tot_5-7 [W]'] = power_av_5_7.sum(axis=1) / shs_sales_volumes['tot_5-7'].values

SHS_POWER_CATEGORIES = shs_power_categories.set_index('category')
SHS_SALES_VOLUMES = shs_sales_volumes.set_index('region')

In [ ]:
SHS_SALES_VOLUMES

In [ ]:
# Global parameters
# variable
tier
invest_shs
invest_mg
# fixed
weight_rise
weight_experts

## Raw data loading

In [ ]:
df = pd.read_csv('daten/raw_data.csv')

In [ ]:
df.head()

## Prepare endogenous results 

In [ ]:
GRID = 'grid'
MG = 'mg'
SHS = 'shs'

ELECTRIFICATION_OPTIONS = [GRID, MG, SHS]
BAU_SENARIO = 'bau'
SE4ALL_SHIFT_SENARIO = 'se4all_shift'
PROG_SENARIO = 'prog'
SENARII = [BAU_SENARIO, SE4ALL_SHIFT_SENARIO, PROG_SENARIO]

MIN_TIER_LEVEL = 3
MIN_RATED_CAPACITY = {3: 200, 4: 800, 5: 2000} # index is TIER level [W]
MIN_ANNUAL_CONSUMPTION = {3: 365, 4: 1250, 5: 3000} # index is TIER level [kWh/a]

# drives for the socio-economic model
MENTI = pd.DataFrame({MG: [3, 13. / 6, 19. / 6, 3.25, 11. / 3],
                    SHS: [23. / 12, 4.5, 37. / 12, 17. / 6, 41. / 12],
                   'labels': ['high_gdp', 'high_mobile_money', 'high_ease_doing_business', 'low_corruption', 'high_grid_weakness']
                   })
MENTI = MENTI.set_index('labels')

In [ ]:
from daten.data_preparation import map_tier_yearly_consumption, get_peak_capacity_from_yearly_consumption

# compute the grid and mg yearly consumption adjusted for tier level
for opt in [GRID, MG]:
    df['hh_%s_tier_yearly_electricity_consumption' % opt] = np.vectorize(map_tier_yearly_consumption)(
        df.hh_yearly_electricity_consumption, 
        df['hh_%s_share' % opt]
    )

#shs_unit_av_capacity  # SHS average capacity [Wp per sold unit]
#=WENN(C3="SSA";$'SHS sales volume'.$V$12;WENN(C3="DA";$'SHS sales volume'.$V$17;WENN(C3="LA";$'SHS sales volume'.$V$20;0)))
df['shs_unit_av_capacity'] = df.region.map(lambda region: SHS_SALES_VOLUMES.loc[region]['weighted_tot_5-7 [W]'])

opt = GRID
#=WENN(AA3<=$Tiers.$C$5;$Tiers.$C$4;WENN(AA3<=$Tiers.$D$5;$Tiers.$D$4))/1000
for tier_level in [5, 4]:
    df.loc[df.hh_yearly_electricity_consumption <= MIN_ANNUAL_CONSUMPTION[tier_level], 'cap_sn2_%s_tier_up' % opt] = MIN_RATED_CAPACITY[tier_level] / 1000

opt = MG
#=$'Data_+_GIS'.T3*$'Data_+_GIS'.AF3
df['cap_sn2_%s_tier_up' % opt] = df.cap_sn2_grid_tier_up * df.hh_mg_share

opt = SHS
#=WENN(V3<=$'SHS sales volume'.$D$6;$'SHS sales volume'.$D$7;$'SHS sales volume'.$D$8)
def shs_av_power(power_cat):
    return SHS_POWER_CATEGORIES.loc[power_cat, 'power_av']

df['cap_sn2_%s_tier_up' % opt] = df.shs_unit_av_capacity.map(lambda shs_cap: shs_av_power(6) if shs_cap <= shs_av_power(5) else shs_av_power(7))

# peak demand
for opt in [GRID, MG]:
    df['hh_%s_tier_peak_demand' % opt] = df['hh_%s_tier_yearly_electricity_consumption' % opt].map(
        get_peak_capacity_from_yearly_consumption, 
        na_action='ignore'
    )

df['pop_rel_growth'] = df.pop_2030 / df.pop_2017
df['pop_dark_2017'] = df.pop_2017 * df.dark_rate
df['pop_newly_electrified_2030'] = df.pop_rel_growth * df.pop_dark_2017
df['pop_electrified_2017'] = df.electrification_rate * df.pop_2017
    

## variables that are not shared between prOG and se4all+SHIFT

## BaU senario data

In [ ]:
bau_data = pd.read_csv('daten/bau.csv', comment='#')
bau_data = bau_data.set_index('region')

df['iea_regional_electricity_coverage'] = df['region'].map(lambda region: bau_data.loc[region]['iea_regional_electricity_coverage'])
df['bau_pop_newly_electrified'] = df.iea_regional_electricity_coverage * df.pop_newly_electrified_2030

for opt in ELECTRIFICATION_OPTIONS:
    #assign the regional electricity share for each options
    df['bau_pop_%s_share' % opt] = df['region'].map(lambda region: bau_data.loc[region]['%s_share' % opt])

## se4all+SHIFT senario data

In [ ]:
from daten.data_preparation import (
    map_weak_grid_class,
    map_corruption_class,
    map_ease_doing_business_class,
    map_gdp_class,
    map_mobile_money_class
)

df['weak_grid_class'] = df['weak_grid_index'].map(map_weak_grid_class, na_action='ignore')
df['corruption_class'] = df['corruption_index'].map(map_corruption_class, na_action='ignore')
df['ease_doing_business_class'] = df['ease_doing_business_index'].map(map_ease_doing_business_class, na_action='ignore')
df['gdp_class'] = df['gdp_per_capita'].map(map_gdp_class, na_action='ignore')
df['mobile_money'] = df['mobile_money_2017'].fillna(df['mobile_money_2014'])
df['mobile_money_class'] = df['mobile_money'].map(map_mobile_money_class, na_action='ignore').fillna(0)

# apply the shift drives
for opt in [MG, SHS]:
    df['shift_menti_%s' % opt] = df.gdp_class * MENTI[opt]['high_gdp'] \
    + df.mobile_money_class * MENTI[opt]['high_mobile_money']
    + df.ease_doing_business_class * MENTI[opt]['high_ease_doing_business']
    + df.corruption_class * MENTI[opt]['low_corruption']
    + df.weak_grid_class * MENTI[opt]['high_grid_weakness']

In [ ]:
# for se4all+SHIFT

WEIGHT_GRID = 0.8  # $RT_shift_factors.$O$2
WEIGHT = 0.2  # $RT_shift_factors.$P$2
RISE_INDICES = ['rise_%s' % opt for opt in ELECTRIFICATION_OPTIONS]
SHIFT_MENTI = ['shift_menti_mg', 'shift_menti_shs']

#SUMME(S22:W22) --> df.loc[:,RISE_INDICES + SHIFT_MENTI].sum(axis=1)
#SUMME($S22:$U22) --> df.loc[:,RISE_INDICES].sum(axis=1)
#SUMME($V22:$W22) --> df.loc[:,SHIFT_MENTI].sum(axis=1)

for opt in ELECTRIFICATION_OPTIONS:
    df['pop_get_%s_2030' % opt] = df['pop_%s_share' % opt] * df.pop_newly_electrified_2030

# to normalize the senarii weigthed sum
weighted_norm = df.loc[:,RISE_INDICES].sum(axis=1) * WEIGHT_GRID + df.loc[:,SHIFT_MENTI].sum(axis=1) * WEIGHT

non_zero_indices = df.loc[:,RISE_INDICES + SHIFT_MENTI].sum(axis=1) != 0

for col in ['shift_grid_share','shift_grid_to_mg_share', 'shift_grid_to_shs_share']:
    # if the sum of the RISE indices and shift MENTI is 0 the corresponding rows in the given columns are set to 0
    df.loc[df.loc[:,RISE_INDICES + SHIFT_MENTI].sum(axis=1) == 0, col] = 0

# share of population which will be on the grid in the se4all+SHIFT senario
#=WENN(SUMME(S22:W22)=0;0;S22*$RT_shift_factors.$O$2/(SUMME($S22:$U22)*$RT_shift_factors.$O$2+SUMME($V22:$W22)*$RT_shift_factors.$P$2))
df.loc[non_zero_indices, 'shift_grid_share'] = df.rise_grid * WEIGHT_GRID / weighted_norm

# share of population which will have changed from grid to mg in the se4all+SHIFT senario
#=WENN(SUMME(S22:W22)=0;0;(T22*$RT_shift_factors.$O$2+V22*$RT_shift_factors.$P$2)/(SUMME($S22:$U22)*$RT_shift_factors.$O$2+SUMME($V22:$W22)*$RT_shift_factors.$P$2))
df.loc[non_zero_indices, 'shift_grid_to_mg_share'] = (df.rise_mg * WEIGHT_GRID + df.shift_menti_mg * WEIGHT ) / weighted_norm

# share of population which will have changed from grid to shs in the se4all+SHIFT senario
#=WENN(SUMME(S23:W23)=0;0;(U23*$RT_shift_factors.$O$2+W23*$RT_shift_factors.$P$2)/(SUMME($S23:$U23)*$RT_shift_factors.$O$2+SUMME($V23:$W23)*$RT_shift_factors.$P$2))
df.loc[non_zero_indices, 'shift_grid_to_shs_share'] = (df.rise_shs * WEIGHT_GRID + df.shift_menti_shs * WEIGHT ) / weighted_norm


# SHARED WITH prOG

#=WENN(Y24>=X24;$Y24*D24;0)
#D24 -->  people get grid until 2030 (=SVERWEIS(B4;$'Data_+_GIS'.$B$3:$V$54;7;0)*SVERWEIS(B4;$'Data_+_GIS'.$B$3:$V$54;13;0))
# if the predicted mg share is larger than the predicted grid share, the number of people predited to use mg in the se4all+SHIFT senario is returned
# otherwise it is set to 0
df.loc[df.shift_grid_to_mg_share >= df.shift_grid_share, 'shift_pop_grid_to_mg'] = df.shift_grid_to_mg_share * df.pop_get_grid_2030
df.loc[df.shift_grid_to_mg_share < df.shift_grid_share, 'shift_pop_grid_to_mg'] = 0

#=WENN(Z25>=X25;Z25*$D25;0)
# if the predicted shs share is larger than the predicted grid share, the number of people predited to use shs in the se4all+SHIFT senario is returned
# otherwise it is set to 0
df.loc[df.shift_grid_to_shs_share >= df.shift_grid_share, 'shift_pop_grid_to_shs'] = df.shift_grid_to_shs_share * df.pop_get_grid_2030
df.loc[df.shift_grid_to_shs_share < df.shift_grid_share, 'shift_pop_grid_to_shs'] = 0

In [ ]:
# for prOG

df.rise_mg = 100
df.rise_shs = 100

weighted_norm = df.loc[:,RISE_INDICES].sum(axis=1)

non_zero_indices = df.loc[:,RISE_INDICES].sum(axis=1) != 0

for col in ['shift_grid_share','shift_grid_to_mg_share', 'shift_grid_to_shs_share']:
    # if the sum of the RISE indices and shift MENTI is 0 the corresponding rows in the given columns are set to 0
    df.loc[df.loc[:,RISE_INDICES].sum(axis=1) == 0, col] = 0

# share of population which will be on the grid in the se4all+SHIFT senario
#=WENN(SUMME(H5;I5;J5)=0;0;H5/SUMME($G5:$I5)) 
df.loc[non_zero_indices, 'shift_grid_share'] = df.rise_grid / weighted_norm

# share of population which will have changed from grid to mg in the se4all+SHIFT senario
#=WENN(SUMME(I5;J5;K5)=0;0;I5/SUMME($G5:$I5))
df.loc[non_zero_indices, 'shift_grid_to_mg_share'] = df.rise_mg / weighted_norm

# share of population which will have changed from grid to shs in the se4all+SHIFT senario
#=WENN(SUMME(I5;J5;K5)=0;0;I5/SUMME($G5:$I5))
df.loc[non_zero_indices, 'shift_grid_to_shs_share'] = df.rise_shs  / weighted_norm

# Shared with se4all

#=WENN(Y24>=X24;$Y24*D24;0)
#D24 -->  people get grid until 2030 (=SVERWEIS(B4;$'Data_+_GIS'.$B$3:$V$54;7;0)*SVERWEIS(B4;$'Data_+_GIS'.$B$3:$V$54;13;0))
# if the predicted mg share is larger than the predicted grid share, the number of people predited to use mg in the se4all+SHIFT senario is returned
# otherwise it is set to 0
df.loc[df.shift_grid_to_mg_share >= df.shift_grid_share, 'shift_pop_grid_to_mg'] = df.shift_grid_to_mg_share * df.pop_get_grid_2030
df.loc[df.shift_grid_to_mg_share < df.shift_grid_share, 'shift_pop_grid_to_mg'] = 0

#=WENN(Z25>=X25;Z25*$D25;0)
# if the predicted shs share is larger than the predicted grid share, the number of people predited to use shs in the se4all+SHIFT senario is returned
# otherwise it is set to 0
df.loc[df.shift_grid_to_shs_share >= df.shift_grid_share, 'shift_pop_grid_to_shs'] = df.shift_grid_to_shs_share * df.pop_get_grid_2030
df.loc[df.shift_grid_to_shs_share < df.shift_grid_share, 'shift_pop_grid_to_shs'] = 0




## Exogenous results for all senarii

In [ ]:
# exogenous results for BaU and se4all+SHIFT

def extract_results_senario(input_df, senario, regions=None):
    
    df = input_df.copy()
    
    if regions is None:
        regions=['SSA', 'DA', 'LA']

        
    if senario == BAU_SENARIO:
        for opt in ELECTRIFICATION_OPTIONS:
            # not valid for other senario than bau at the moment
            # create a columns with regional electrification option shares 
            df['temp_%s' % opt] = df['region'].replace(regions, bau_data.loc[regions]['%s_share' % opt].to_list())

            # predicted number of people getting access to electricity (regional detail level)
            df['pop_get_%s_2030' % opt] = df.bau_pop_newly_electrified * df['temp_%s' % opt]
    elif senario in [SE4ALL_SHIFT_SENARIO, PROG_SENARIO]:
        #SUMME(AA4:AB4) --> df.loc[:,['shift_pop_grid_to_mg' 'shift_pop_grid_to_shs']].sum(axis=1) 
        #grid =D4-SUMME(AA4:AB4)
        opt = 'grid'
        # predicted number of people getting access to electricity (regional detail level)
        df['pop_get_%s_2030' % opt] = df['pop_get_%s_2030' % opt] - df.loc[:,['shift_pop_grid_to_mg', 'shift_pop_grid_to_shs']].sum(axis=1) 

        #mg =E5+AA5
        opt = 'mg'
        # predicted number of people getting access to electricity (regional detail level)
        df['pop_get_%s_2030' % opt] = df['pop_get_%s_2030' % opt] + df['shift_pop_grid_to_%s' % opt]

        #shs =F6+AB6
        opt = 'shs'
        # predicted number of people getting access to electricity (regional detail level)
        df['pop_get_%s_2030' % opt] = df['pop_get_%s_2030' % opt] + df['shift_pop_grid_to_%s' % opt]
    else:
        raise(ValueError)

    for opt in ELECTRIFICATION_OPTIONS:
        # predicted number of household getting access to electricity (regional detail level)
        df['hh_get_%s_2030' % opt] = df['pop_get_%s_2030' % opt] / df.hh_av_size
        # predicted power (in kW) that the access to electricity will represent (regional detail level)
        # the analysis is based on the peak demand for the grid and mg senarii, and the average
        # power of solar panel for shs senario
        if opt in ('grid' , 'mg'):
            df['hh_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['hh_%s_tier_peak_demand' % opt]
            df['hh_cap_scn2_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['cap_sn2_%s_tier_up' % opt]
        else:
            df['hh_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['shs_unit_av_capacity'] / 1000
            df['hh_cap_scn2_%s_capacity' % opt] = df['hh_get_%s_2030' % opt] * df['cap_sn2_%s_tier_up' % opt] / 1000
            
        

In [ ]:
extract_results_senario(df, SE4ALL_SHIFT_SENARIO)

In [ ]:
extract_results_senario(df, BAU_SENARIO)

In [ ]:
extract_results_senario(df, PROG_SENARIO)

In [ ]:
#=WENN(Q22<4,5;0;WENN(Q22<=9;0,5;WENN(Q22>9;1;"NA")))

def map_weak_grid_class(weak_grid_idx):
    """Assign an index value to differentiate weak grid"""
    answer = 1
    if weak_grid_idx <= 4.5:
        answer = 0.5
    if weak_grid_idx <= 9:
        answer = 0
    return answer

#=WENN(O25<26;0;WENN(O25<=33;0,5;WENN(O25>33;1;"NA")))

def map_corruption_class(corruption_idx):
    """Assign an index value to differentiate corruption"""
    answer = 1
    if corruption_idx <= 33:
        answer = 0.5
    if corruption_idx <= 26:
        answer = 0
    return answer

#=WENN(M22<131;0;WENN(M22<=164;0,5;WENN(M22>164;1;"NA")))

def map_ease_doing_business_class(business_ease):
    """Assign an index value to differentiate ease of doing business"""
    answer = 1
    if business_ease <= 164:
        answer = 0.5
    if business_ease <= 131:
        answer = 0
    return answer

#Se4All

def map_gdp_class(gdp_per_capita):
    """Assign an index value to differentiate gdp per capita"""
    answer = 1
    if gdp_per_capita < 1500:
        answer = 0.5
    if gdp_per_capita < 700:
        answer = 0
    return answer
    
#=WENN(K5<=0,12;0;WENN(K5<=0,21;0,5;1))

def map_mobile_money_class(mobile_money):
    """Assign an index value to differentiate mobile_money"""
    answer = 1
    if mobile_money <= 0.21:
        answer = 0.5
    if mobile_money <= 0.12:
        answer = 0
    return answer


In [ ]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                    'B': [5, -1, 7, -3, 9],
                   'C': [4, 7, 3, 9, 1],
                   'D': [1, 5, 3, 0, 1],
                   'E': [7, 7, 2, 9, 7],
                    'Z': ['a', 'a', 'b', 'c', 'c']})
WEIGHT_GRID = 0.8  # $RT_shift_factors.$O$2
WEIGHT = 0.2  # $RT_shift_factors.$P$2



RISE_INDICES = ['A', 'B']
SHIFT_MENTI = []
# norm = df.loc[:,RISE_INDICES].sum(axis=1) * WEIGHT_GRID + df.loc[:,SHIFT_MENTI].sum(axis=1) * WEIGHT
# df['E'] * WEIGHT_GRID / norm

entries = ['shift_grid_share','shift_grid_to_mg_share', 'shift_grid_to_shs_share']
# df.loc[df.loc[:,RISE_INDICES + SHIFT_MENTI].sum(axis=1) == 0, 'shift_grid_share' ] = '0'
for entry in entries:
    df.loc[df.B >0, entry ] = '0'

df